In [ ]:
import cv2
import numpy as np
import time
from IPython.display import clear_output
import math
import requests

WIND_X = 800
WIND_Y = 600

GOAL_POS = 100
GOAL_LEN = 300

FIELD_COLOR = (10,  240, 10)
GOAL_COLOR  = (30,   40, 70)
BALL_COLOR  = (120, 130, 30)
ROBOT_COLOR = (160,  40, 10)
TRAJ_COLOR  = (60,   40, 180)

BALL_RADIUS   = 20
CIRCLE_RADIUS = 50
ROBOT_SIZE    = 50

def draw_scenario (xr, yr, xb, yb, trajectory):
    canvas = np.zeros ((WIND_Y, WIND_X, 3), np.uint8)
    
    cv2.rectangle (canvas, (0, 0), (WIND_X, WIND_Y), FIELD_COLOR, thickness = -1)
    
    cv2.circle    (canvas, (xb, yb), CIRCLE_RADIUS, GOAL_COLOR)
    cv2.circle    (canvas, (xb, yb), BALL_RADIUS, BALL_COLOR, -1)
    
    robot_tl = (xr - int (ROBOT_SIZE / 2), yr - int (ROBOT_SIZE / 2))
    robot_br = (xr + int (ROBOT_SIZE / 2), yr + int (ROBOT_SIZE / 2))
    cv2.rectangle (canvas, robot_tl, robot_br, GOAL_COLOR, -1)
    
    cv2.line (canvas, (WIND_X, GOAL_POS), (WIND_X, GOAL_POS + GOAL_LEN), GOAL_COLOR, 5)
    
    for i in range (len (trajectory) - 1):
        cv2.line (canvas, trajectory [i], trajectory [i + 1], TRAJ_COLOR, 3)
    
    return canvas

def find_trajectory_test (xr, yr, xb, yb):
    traj = []
    
    traj.append ((xr, yr))
    traj.append ((xb, yb))
    
    return traj

def find_trajectory_basic (xr, yr, xb, yb, max_step):
    traj = []
    
    traj.append ((xr, yr))
    
    #find starting point on the circle
    rbx = xb - xr #robot-ball x
    rby = yb - yr #robot-ball y
    
    length_rb = int (math.sqrt (rbx**2 + rby**2))
    
    spx = xb - int (CIRCLE_RADIUS * rbx / length_rb) #start point x
    spy = yb - int (CIRCLE_RADIUS * rby / length_rb) #start point y
    
    if (abs (xb - xr) < abs (yb - yr)):
        traj.append ((spx, spy))
    
    #find kick point on the circle
    gbx = xb - WIND_X                        #goal-ball x
    gby = yb - GOAL_POS - int (GOAL_LEN / 2) #goal-ball y
    
    length_gb = int (math.sqrt (gbx**2 + gby**2))
    
    kpx = xb + int (CIRCLE_RADIUS * gbx / length_gb) #kick point x
    kpy = yb + int (CIRCLE_RADIUS * gby / length_gb) #kick point y
    
    traj.append ((kpx, kpy))
    traj.append ((WIND_X, GOAL_POS + int (GOAL_LEN / 2)))
        
    return traj

def find_trajectory (xr, yr, xb, yb, max_step):
    traj = []
    
    traj.append ((xr, yr))
    
    #-----------------------------------------------------------
    #find starting point on the circle
    
    xbr = xb - xr #x ball relative
    ybr = yb - yr #y ball relative
    
    r    = CIRCLE_RADIUS
    leng = math.sqrt (xbr**2 + ybr**2)
    
    beta  = np.arcsin (float (ybr) / leng)
    alpha = np.arcsin (float (r) / leng)
    
    if (xb < xr):# + CIRCLE_RADIUS):
        sx = 0
        sy = 0
        
        if (yr + (yr - GOAL_POS - int (GOAL_LEN / 2)) * (xr - xb) / (WIND_X - xb) > yb):
            sx = - leng * math.cos (alpha + beta) * math.cos (alpha) + xr
            sy = leng * math.sin (alpha + beta) * math.cos (- alpha) + yr
        
        else:
            alpha = - alpha
            
            sx = - leng * math.cos (alpha + beta) * math.cos (alpha) + xr
            sy = leng * math.sin (alpha + beta) * math.cos (- alpha) + yr
        
        traj.append ((int (sx), int (sy)))
    
    #-----------------------------------------------------------
    #find kick point on the circle
    gbx = xb - WIND_X                        #goal-ball x
    gby = yb - GOAL_POS - int (GOAL_LEN / 2) #goal-ball y
    
    length_gb = int (math.sqrt (gbx**2 + gby**2))
    
    kpx = xb + int (CIRCLE_RADIUS * gbx / length_gb) #kick point x
    kpy = yb + int (CIRCLE_RADIUS * gby / length_gb) #kick point y
    
    traj.append ((kpx, kpy))
    
    #move with proper steps on the circle
    
    #if (xr < xb):
    #    traj.append ((xr, yr))
    #    traj.append ((xb - CIRCLE_RADIUS, yb))
    
    #else:
    #    if (yr < yb):
    #    else:
    
    traj.append ((WIND_X, GOAL_POS + int (GOAL_LEN / 2)))
    
    return traj

cv2.namedWindow  ("movement", cv2.WINDOW_AUTOSIZE)
cv2.resizeWindow ("movement", (WIND_Y, WIND_X))

#ip   = "http://192.168.1.29:"
#port = "9570"

ip   = "http://10.0.1.178:"
port = "9560"

xr = int (1 * WIND_X / 5)
yr = int (WIND_Y / 3)

xb = int (WIND_X / 2)
yb = int (WIND_Y / 2)

upd = True

trajectory = []

canvas = np.ones ((WIND_Y, WIND_X, 3), np.uint8) * 100

while (True):    
    cv2.waitKey (1)
    
    #if (upd == True):
    #    trajectory = find_trajectory (xr, yr, xb, yb, 20)
    #    upd = False
    
    #canvas = draw_scenario (xr, yr, xb, yb, trajectory)
        
    cv2.imshow ("movement", canvas)
    
    time.sleep  (0.01)
    
    #handle keyboard events
    keyb = cv2.waitKey (1)
    
    if (keyb != -1):
        upd = True
    
    if (keyb & 0xFF == ord ('q')):
        break

    elif (keyb & 0xFF == ord ('t')):
        yr -= 5
    
    ####################################
    
    elif (keyb & 0xFF == ord ('z')):
        r = requests.get (ip + port + "/?action=/stand&text=qwer")
    
    elif (keyb & 0xFF == ord ('x')):
        r = requests.get (ip + port + "/?action=/sit&text=qwer")
    
    elif (keyb & 0xFF == ord ('c')):
        r = requests.get (ip + port + "/?action=/rest&text=qwer")

    ######################################
    
    elif (keyb & 0xFF == ord ('i')):
        r = requests.get (ip + port + "/?action=/walk_20&text=qwer")
            
    elif (keyb & 0xFF == ord ('u')):
        r = requests.get (ip + port + "/?action=/walk_50&text=qwer")

    elif (keyb & 0xFF == ord ('m')):
        r = requests.get (ip + port + "/?action=/walk_m30&text=qwer")
        
    elif (keyb & 0xFF == ord ('h')):
        r = requests.get (ip + port + "/?action=/rot_50&text=qwer")

    elif (keyb & 0xFF == ord ('j')):
        r = requests.get (ip + port + "/?action=/rot_20&text=qwer")

    elif (keyb & 0xFF == ord ('k')):
        r = requests.get (ip + port + "/?action=/rot_m20&text=qwer")

    elif (keyb & 0xFF == ord ('l')):
        r = requests.get (ip + port + "/?action=/rot_m50&text=qwer")

    ######################################

    elif (keyb & 0xFF == ord ('a')):
        r = requests.get (ip + port + "/?action=/red&text=qwer")

    elif (keyb & 0xFF == ord ('s')):
        r = requests.get (ip + port + "/?action=/green&text=qwer")

    elif (keyb & 0xFF == ord ('d')):
        r = requests.get (ip + port + "/?action=/blue&text=qwer")

    elif (keyb & 0xFF == ord ('f')):
        r = requests.get (ip + port + "/?action=/right_hand_front&text=qwer")

    elif (keyb & 0xFF == ord ('g')):
        r = requests.get (ip + port + "/?action=/hands&text=open_right")

    elif (keyb & 0xFF == ord ('b')):
        r = requests.get (ip + port + "/?action=/hands&text=close_right")

    #elif (keyb & 0xFF == ord ('n')):
    #    r = requests.get (ip + port + "/?action=/lightblue&text=close_right")

    #elif (keyb & 0xFF == ord ('1')):
    #    r = requests.get (ip + port + "/?action=/pink&text=close_right")

    elif (keyb & 0xFF == ord ('n')):
        r = requests.get (ip + port + "/?action=/set_right_hand_angle&text=close_right")

cv2.waitKey           (1)
cv2.destroyAllWindows ()